# PIP INSTALL AND IMPORT MODULE

In [ ]:
!pip install datasets evaluate transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup

from datasets import DatasetDict, Dataset
from sklearn.model_selection import train_test_split
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import ast

# We can see all columns in df.head() / and .tail()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_colwidth', None)

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#this link
file_id = '1i5-lhcT6N2IOhTmKmfTaJmvSSQkgsFKA'

#298A link
# file_id = '1k6vFnOwLBB_wlWOL57lkzorqQ2Q89XG_' #<-- You add in here the id from you google drive file, you can find it
download = drive.CreateFile({'id': file_id})

# Download the file to a local disc
download.GetContentFile('tokenized_df.csv')

# PREPROCESSING


In [ ]:
df = pd.read_csv('tokenized_df.csv', engine='python')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546785 entries, 0 to 546784
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   content_text    546785 non-null  object
 1   input_ids       546785 non-null  object
 2   attention_mask  546785 non-null  object
 3   targets         546785 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 16.7+ MB


In [ ]:
df.head()

,content_text,input_ids,attention_mask,targets
0,Wall St. Pullback Reflects Tech Blowout (Reute...,"[101, 2813, 2358, 1012, 4139, 5963, 11138, 662...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
1,Wall St. Bears Claw Back Into the Black (Reute...,"[101, 2813, 2358, 1012, 6468, 15020, 2067, 204...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,"[101, 3514, 1998, 4610, 6112, 15768, 1005, 176...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
3,Iraq Halts Oil Exports from Main Southern Pipe...,"[101, 5712, 9190, 2015, 3514, 14338, 2013, 236...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
4,"Stocks End Up, But Near Year Lows (Reuters) Re...","[101, 15768, 2203, 2039, 1010, 2021, 2379, 209...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0


In [ ]:
df.rename(columns={'content_text': 'text', 'targets':'label'}, inplace=True)
df = df[['text', 'label', 'input_ids', 'attention_mask']]

In [ ]:
df['input_ids'] = df['input_ids'].apply(lambda x: ast.literal_eval(x))
df['attention_mask'] = df['attention_mask'].apply(lambda x: ast.literal_eval(x))

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)
train_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)
val_df.reset_index(drop=True,inplace=True)

In [ ]:
train_df.head()

,text,label,input_ids,attention_mask
0,Wall St. Pullback Reflects Tech Blowout (Reute...,0,"[101, 2813, 2358, 1012, 4139, 5963, 11138, 662...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Wall St. Bears Claw Back Into the Black (Reute...,0,"[101, 2813, 2358, 1012, 6468, 15020, 2067, 204...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,0,"[101, 3514, 1998, 4610, 6112, 15768, 1005, 176...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Iraq Halts Oil Exports from Main Southern Pipe...,0,"[101, 5712, 9190, 2015, 3514, 14338, 2013, 236...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"Stocks End Up, But Near Year Lows (Reuters) Re...",0,"[101, 15768, 2203, 2039, 1010, 2021, 2379, 209...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
val_df.head()

,text,label,input_ids,attention_mask
0,Census Bureau admits privacy breach (AP) ...,1,"[101, 2883, 4879, 14456, 9394, 12510, 1006, 97...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"Sony to show ""Home,"" ends wait on PS3 Web stra...",1,"[101, 8412, 2000, 2265, 1000, 2188, 1010, 1000...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Worldbeat: Now it's the ads that zap viewers ...,1,"[101, 2088, 19442, 1024, 2085, 2009, 1005, 105...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Video game: `Crackdown' packs a punch (AP...,1,"[101, 2678, 2208, 1024, 1036, 8579, 7698, 1005...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Early birds: Why small was beautiful for dino ...,1,"[101, 2220, 5055, 1024, 2339, 2235, 2001, 3376...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
test_df.head()

,text,label,input_ids,attention_mask
0,Best Buy Profits Rise 8.7 Percent MINNEAPOL...,1,"[101, 2190, 4965, 11372, 4125, 1022, 1012, 102...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Times Ends Fee on TimesSelect Content New Y...,1,"[101, 2335, 4515, 7408, 2006, 2335, 11246, 224...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,DHS 'Dry Run' Support Cited In the weeks be...,1,"[101, 28144, 2015, 1005, 4318, 2448, 1005, 249...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Lehman net falls on writedowns NEW YORK (Re...,0,"[101, 28444, 5658, 4212, 2006, 4339, 7698, 201...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Health Care Spending Highest in Northeast W...,0,"[101, 2740, 2729, 5938, 3284, 1999, 4794, 2899...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
train_df.head()

,text,label,input_ids,attention_mask
0,Wall St. Pullback Reflects Tech Blowout (Reute...,0,"[101, 2813, 2358, 1012, 4139, 5963, 11138, 662...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,Wall St. Bears Claw Back Into the Black (Reute...,0,"[101, 2813, 2358, 1012, 6468, 15020, 2067, 204...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,0,"[101, 3514, 1998, 4610, 6112, 15768, 1005, 176...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Iraq Halts Oil Exports from Main Southern Pipe...,0,"[101, 5712, 9190, 2015, 3514, 14338, 2013, 236...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"Stocks End Up, But Near Year Lows (Reuters) Re...",0,"[101, 15768, 2203, 2039, 1010, 2021, 2379, 209...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
train = Dataset.from_pandas(train_df)
test = Dataset.from_pandas(test_df)
validation = Dataset.from_pandas(val_df)
transformed_dataset = DatasetDict({'train': train, 'validation': validation,'test': test})

In [ ]:
print(transformed_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 382749
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 82018
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 82018
    })
})


In [ ]:
#map to map target numeric w actual values
id2label = {0:"Business", 1:"Sci/Tech", 2:"Entertainment", 3:"Sports", 4:"World"}
label2id = {"Business":0, "Sci/Tech":1, "Entertainment":2, "Sports":3, "World":4}

# COMPUTE METRIC FUNCTION

In [ ]:
from datasets import load_metric

accuracy_metric = load_metric("accuracy")
precision_metric = load_metric("precision")
recall_metric = load_metric("recall")
f1_metric = load_metric("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")


    return {"precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": accuracy}

<ipython-input-11-112a9d4891aa>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")


# TRAINING

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilroberta-base", num_labels=5, id2label=id2label, label2id=label2id
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
transformed_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 382749
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 82018
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 82018
    })
})

In [ ]:
training_args = TrainingArguments(
    output_dir="roberta-base-agnews",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=transformed_dataset["train"],
    eval_dataset=transformed_dataset["validation"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.540700,0.538807,{'precision': 0.7953485832124483},{'recall': 0.7968616645126679},{'f1': 0.7927541955103132},{'accuracy': 0.7968616645126679}
2,0.443200,0.484345,{'precision': 0.8150332070178398},{'recall': 0.8174547050647419},{'f1': 0.8141768442633359},{'accuracy': 0.8174547050647419}
3,0.420500,0.470900,{'precision': 0.8223252051650776},{'recall': 0.8245263234899656},{'f1': 0.8211243338399905},{'accuracy': 0.8245263234899656}


Trainer is attempting to log a value of "{'precision': 0.7953485832124483}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.7968616645126679}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.7927541955103132}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.7968616645126679}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8150332070178398}" of type <class

TrainOutput(global_step=71766, training_loss=0.5048699374847799, metrics={'train_runtime': 11451.9465, 'train_samples_per_second': 100.267, 'train_steps_per_second': 6.267, 'total_flos': 2.9709654462981e+16, 'train_loss': 0.5048699374847799, 'epoch': 3.0})